In [1]:
# Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Load and Preprocess Data
train_df = pd.read_csv('train.csv').drop(columns=['Unnamed: 0'])
test_df = pd.read_csv('test.csv').drop(columns=['Unnamed: 0'])
merged_train_df = pd.concat([train_df, test_df['Survived']], axis=1)
X = merged_train_df.drop(columns=['Survived'])
y = merged_train_df['Survived']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [3]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train.values).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test.values).view(-1, 1)


In [4]:
# DataLoader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [5]:
# Define 3-Layer Neural Network
class ThreeLayerNN(nn.Module):
    def __init__(self, input_dim):
        super(ThreeLayerNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [6]:
# Initialize Model, Loss, and Optimizer
model = ThreeLayerNN(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Training Loop
epochs = 50
for epoch in range(epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

In [8]:
# Evaluation
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor)
    test_loss = criterion(y_test_pred, y_test_tensor)
    y_test_pred_class = (y_test_pred > 0.5).float()
    test_accuracy = (y_test_pred_class == y_test_tensor).float().mean()

In [9]:
print(f'Test Accuracy: {test_accuracy.item()}')

Test Accuracy: 0.826815664768219
